### 경사하강법과  로지스틱 회귀

#### 필요한 패키지를 읽어 온다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

#### 데이터 읽어오기.

In [ ]:
df = pd.read_csv('data_mtcars.csv', header='infer',encoding='latin1')
df.head(5)

In [ ]:
X = np.array(df.drop(columns=['am','model']))
Y = np.array(df.loc[:,'am'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=3)    

#### 시그모이드 함수와 그래디언트 함수.

In [ ]:
def sigmoid(x):
    s = 1.0/(1.0 + np.exp(-x))
    return s

def gradient(X, Y, beta):
    z = np.dot(X,beta.T)*Y
    ds = -Y*(1-sigmoid(z))*X
    return ds.sum(axis=0)

#### Logistic Regression 클래스 정의.

In [ ]:
class logisticRegression:
    def __init__(self,n_features,learn_rate):
        self.rate = learn_rate
        self.inodes = n_features + 1
        self.beta = np.random.normal(0.0,1.0,(1,self.inodes))
        
    def train(self, input_X, input_Y, n_epochs):
        ones_column = np.ones((input_X.shape[0],1))
        X = np.concatenate((ones_column,input_X),axis=1)
        Y = (2*input_Y - 1).reshape(-1,1)
        # 경사하강법으로 계수를 구한다!
        for n in range(n_epochs):                               
            self.beta = self.beta - self.rate*gradient(X,Y,self.beta)
        return self.beta
    
    def query(self, input_X, prob=True, thresh=0.5):
        ones_column = np.ones((input_X.shape[0],1))
        X = np.concatenate((ones_column,input_X),axis=1)
        z = np.dot(X,(self.beta).T)
        p = sigmoid(z)
        if prob :
            return p
        else:
            return (p > thresh).astype('int')

#### Logistic 회귀 객체를 생성하여 학습하고 정확도를 계산한다.

In [ ]:
LR = logisticRegression(X_train.shape[1], 0.01)                       # learning rate = 0.01
LR.train(X_train, Y_train, 2000)                                      # epochs = 2000
Y_forecast = LR.query(X_test,prob=False,thresh=0.5)                   # 예측.
accuracy = (Y_forecast == Y_test.reshape(-1,1)).mean()                # 예측과 실제 값의 일치 비율 = 정확도.
print("Accuracy : {0}".format(accuracy))